## INTENTION OF THIS WORKBOOK
This next pass will calculate the CAGR of holding company tiers (and ALL independents) of the AdAge US Top 900 advertising 

In [1]:
import pandas as pd
import re
%matplotlib inline

In [2]:
# create dataframes for the six years we will be looking at
# by importing the 2010 thru 2015 AdAge data
raw15='/Users/xavier/Documents/src/dataviz/AgencyRevenueModels/adage-data/adage-900_2015.csv'
raw14='/Users/xavier/Documents/src/dataviz/AgencyRevenueModels/adage-data/adage-900_2014.csv'
raw13='/Users/xavier/Documents/src/dataviz/AgencyRevenueModels/adage-data/adage-900_2013.csv'
raw12='/Users/xavier/Documents/src/dataviz/AgencyRevenueModels/adage-data/adage-900_2012.csv'
raw11='/Users/xavier/Documents/src/dataviz/AgencyRevenueModels/adage-data/adage-900_2011.csv'
raw10='/Users/xavier/Documents/src/dataviz/AgencyRevenueModels/adage-data/adage-900_2010.csv'
adage15 = pd.read_csv(raw15)
adage15.name = '2015'
adage14 = pd.read_csv(raw14)
adage14.name = '2014'
adage13 = pd.read_csv(raw13)
adage13.name = '2013'
adage12 = pd.read_csv(raw12)
adage12.name = '2012'
adage11 = pd.read_csv(raw11)
adage11.name = '2011'
adage10 = pd.read_csv(raw10)
adage10.name = '2010'

In [3]:
fiveyear = [adage15,adage14,adage13,adage12,adage11,adage10]
burgeoning_lower_limit = 150000000

In [4]:
# A function to RETURN two variables from AGENCY-COMPANY column
def indySubstituteSimple(x):
    parent = re.compile("\[(.*)\]")
    owned = re.compile("(.+?)\[(.*)\]")
    p = parent.search(x)
    if p:
        o = owned.search(x)
        return o.group(1),o.group(2)
    else:
        return x, "Independent"

In [5]:
# Create two new Columns that take the two outputs of the function
# Remove the bigger agency parent, e.g. 'Omnicom (child of BBDO)'
# Strip any trailing whitespaces in the names
# Transform revenue to the literal number, not 'In thousands'
# Remove the * asterisk after the agency name
def defineTiers(z):
    z['AGENCY-NAME'], z['AGENCY-OWNER'] = zip(*z['AGENCY-COMPANY'].map(indySubstituteSimple))
    z.loc[:, 'AGENCY-OWNER'] = z['AGENCY-OWNER'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))
    z.loc[:, 'AGENCY-OWNER'] = z['AGENCY-OWNER'].apply(lambda x: re.sub(r'\s+$', '', x))
    z.iloc[:,2] = z.iloc[:,2].apply(lambda x: x*1000)
    z.iloc[:,4] = z.iloc[:,4].apply(lambda x: re.sub(r'\*', '', x))
#    z['AGENCY-TIER'] = z['AGENCY-OWNER'].apply(typer)

In [6]:
for i in range (0,len(fiveyear),1):
    defineTiers(fiveyear[i])

In [7]:
# Group the companies by owner
tiers15 = adage15.groupby(adage15.iloc[:,5]).sum().reset_index()
tiers14 = adage14.groupby(adage14.iloc[:,5]).sum().reset_index()
tiers13 = adage13.groupby(adage13.iloc[:,5]).sum().reset_index()
tiers12 = adage12.groupby(adage12.iloc[:,5]).sum().reset_index()
tiers11 = adage11.groupby(adage11.iloc[:,5]).sum().reset_index()
tiers10 = adage10.groupby(adage10.iloc[:,5]).sum().reset_index()

In [8]:
# Supernova CAGR—WITHOUT INDEPENDENTS
sn15 = tiers15[(tiers15.iloc[:,1] > 2000000000) & (tiers15.iloc[:,0] != 'Independent')]
sn10 = tiers10[(tiers10.iloc[:,1] > 2000000000) & (tiers10.iloc[:,0] != 'Independent')]
supernova_cagr = ((sn15.iloc[:,1].sum()/sn10.iloc[:,1].sum())**(1/5))-1
print("Supernova holding companies compound annual growth rate over five years: "+"{:.1%}".format(supernova_cagr))

Supernova holding companies compound annual growth rate over five years: 9.6%


In [9]:
#Double check from Google Sheet
((24049151000/15202010000)**(1/5))-1

0.09607367469576777

In [10]:
supernova_cagr

0.09607367469576777

In [11]:
# Independents CAGR
in15 = tiers15[(tiers15.iloc[:,0] == 'Independent')]
in10 = tiers10[(tiers10.iloc[:,0] == 'Independent')]
in_cagr = ((in15.iloc[:,1].sum()/in10.iloc[:,1].sum())**(1/5))-1
print("Independent agencies compound annual growth rate over five years: "+"{:.1%}".format(in_cagr))

Independent agencies compound annual growth rate over five years: 5.6%


In [12]:
# Midmarket CAGR
mid15 = tiers15[\
    (tiers15.iloc[:,1] <= 1999999999) & \
    (tiers15.iloc[:,1] >= 500000000)]
mid10 = tiers10[\
    (tiers10.iloc[:,1] <= 1999999999) & \
    (tiers10.iloc[:,1] >= 500000000)]
mid_cagr = ((mid15.iloc[:,1].sum()/mid10.iloc[:,1].sum())**(1/5))-1
print("Midmarket holding companies compound annual growth rate over five years: "+"{:.1%}".format(mid_cagr))

Midmarket holding companies compound annual growth rate over five years: 30.2%


In [13]:
# Burgeoning CAGR
bg15 = tiers15[\
    (tiers15.iloc[:,1] <= 499999999) & \
    (tiers15.iloc[:,1] >= burgeoning_lower_limit)]
bg10 = tiers10[\
    (tiers10.iloc[:,1] <= 499999999) & \
    (tiers10.iloc[:,1] >= burgeoning_lower_limit)]
bg_cagr = ((bg15.iloc[:,1].sum()/bg10.iloc[:,1].sum())**(1/5))-1
print("Burgeoning holding companies compound annual growth rate over five years: "+"{:.1%}".format(bg_cagr))

Burgeoning holding companies compound annual growth rate over five years: 1.0%


In [14]:
sn15

,AGENCY-OWNER,2015-REVENUE,% CHG
4,Alliance Data Systems Corp.,2050465000,17.6
22,Interpublic,4451749000,256.7
29,Omnicom,6777123000,323.5
31,Publicis,5186067000,225.5
35,WPP,5583747000,522.8


In [15]:
sn10

,AGENCY-OWNER,2010-REVENUE,% CHG
28,Interpublic,3606889000,289.8
38,Omnicom,4595246000,568.3
40,Publicis,3072242000,370.9
51,WPP,3927633000,510.3


In [16]:
mid15

,AGENCY-OWNER,2015-REVENUE,% CHG
0,Accenture,1231596000,70.8
9,DJE Holdings,558486000,33.7
10,Deloitte,865200000,NaN
11,Dentsu,942850000,159.1
16,Havas,859391000,16.2
19,IBM Corp.,796800000,NaN
24,MDC Partners,1028257000,387.3
32,PwC,624000000,NaN


In [17]:
mid10

,AGENCY-OWNER,2010-REVENUE,% CHG
2,Alliance Data Systems Corp.,584400000,19.8
20,Havas,674687000,205.5
31,MDC Partners,590307000,473.6


In [18]:
bg15

,AGENCY-OWNER,2015-REVENUE,% CHG
1,Acosta,321717000,5.9
3,Advantage Solutions,480668000,9.2
13,Engine Group,177783000,26.2
14,Experian,432000000,-0.2
17,Hearst Corp.,224940000,20.0
20,ICF International,165000000,NaN
30,Project WorldWide,224279000,124.5
36,inVentiv Group Holdings,325844000,8.2


In [19]:
bg10

,AGENCY-OWNER,2010-REVENUE,% CHG
1,Aegis Group,189900000,47.9
9,Daniel J. Edelman,361691000,77.6
10,Dentsu,230800000,170.1
15,GSI Commerce Inc.,162881000,48.8
16,Groupe Aeroplan,266251000,4.0
25,IBM Corp.,233600000,10.9
34,Meredith Corp.,173000000,-2.8
46,Sapient Corp.,350000000,35.1
52,inVentiv Group Holdings,269425000,16.5


In [21]:
(1000000*.06)

60000.0